In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import pdb

class Grid(object):
    def __init__(self, d1, d2, rotate = 1, ratio = 0.5, mode=0, prob=1.):
        self.d1 = d1
        self.d2 = d2
        self.rotate = rotate
        self.ratio = ratio
        self.mode=mode
        self.st_prob = self.prob = prob

    def set_prob(self, epoch, max_epoch):
        self.prob = self.st_prob * min(1, epoch / max_epoch)

    def __call__(self, img):
        if np.random.rand() > self.prob:
            return img
        h = img.size(1)
        w = img.size(2)
        hh = int(1.5*h)
        ww = int(1.5*w)
        d = np.random.randint(self.d1, self.d2)
        #d = self.d
        self.l = int(d*self.ratio+0.5)
        mask = np.ones((hh, ww), np.float32)
        st_h = np.random.randint(d)
        st_w = np.random.randint(d)
        for i in range(-1, hh//d+1):
                s = d*i + st_h
                t = s+self.l
                s = max(min(s, hh), 0)
                t = max(min(t, hh), 0)
                mask[s:t,:] *= 0
        for i in range(-1, ww//d+1):
                s = d*i + st_w
                t = s+self.l
                s = max(min(s, ww), 0)
                t = max(min(t, ww), 0)
                mask[:,s:t] *= 0
        r = np.random.randint(self.rotate)
        mask = Image.fromarray(np.uint8(mask))
        mask = mask.rotate(r)
        mask = np.asarray(mask)
        mask = mask[(hh-h)//2:(hh-h)//2+h, (ww-w)//2:(ww-w)//2+w]

        mask = torch.from_numpy(mask).float().cuda()
        if self.mode == 1:
            mask = 1-mask

        mask = mask.expand_as(img)
        img = img * mask 

        return img

class GridMask(nn.Module):
    def __init__(self, d1, d2, rotate = 1, ratio = 0.6, mode=1, prob=1.):
        super(GridMask, self).__init__()
        self.rotate = rotate
        self.ratio = ratio
        self.mode = mode
        self.st_prob = prob
        self.grid = Grid(d1, d2, rotate, ratio, mode, prob)

    def set_prob(self, epoch, max_epoch):
        self.grid.set_prob(epoch, max_epoch)

    def forward(self, x):
        if not self.training:
            return x
        n,c,h,w = x.size()
        y = []
        for i in range(n):
            y.append(self.grid(x[i]))
        y = torch.cat(y).view(n,c,h,w)
        return y


In [8]:
HEIGHT = 137
WIDTH = 236

In [9]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [10]:
#plt.imshow(x)

In [11]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        #if self.train_mode:
        #    augs = get_train_augs()
        #    img = augs(image=img)['image']
        
        img = np.expand_dims(img, axis=-1)
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [12]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

# model

In [13]:
#import pretrainedmodels

In [14]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [15]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [16]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [17]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

In [18]:
#model.last_linear.in_features

In [19]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        #self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        #self.fix_input_layer()
        
    def fix_input_layer(self):
        if self.backbone_name in ['se_resnext50_32x4d', 'se_resnext101_32x4d', 'se_resnet50', 'senet154', 'se_resnet152', 'nasnetmobile', 'mobilenet', 'nasnetalarge']:
            #self.backbone = eval(backbone_name)()
            #print(self.backbone.layer0.conv1)
            w = self.backbone.layer0.conv1.weight.data
            self.backbone.layer0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            #self.backbone.layer0.conv1.weight = torch.nn.Parameter(torch.cat((w, w[:, 2, :, :].unsqueeze(1)), dim=1))
            self.backbone.layer0.conv1.weight = torch.nn.Parameter(w[:, 0, :, :].unsqueeze(1))
        
    def logits(self, x):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [20]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [21]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [22]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [23]:
round(1/9, 6)

0.111111

In [24]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [25]:
OHEM = True
def ohem_loss(cls_pred, cls_target, rate=0.5):
    batch_size = cls_pred.size(0) 
    ohem_cls_loss = F.cross_entropy(cls_pred, cls_target, reduction='none', ignore_index=-1)

    sorted_ohem_loss, idx = torch.sort(ohem_cls_loss, descending=True)
    keep_num = min(sorted_ohem_loss.size()[0], int(batch_size*rate) )
    if keep_num < sorted_ohem_loss.size()[0]:
        keep_idx_cuda = idx[:keep_num]
        ohem_cls_loss = ohem_cls_loss[keep_idx_cuda]
    cls_loss = ohem_cls_loss.sum() / keep_num
    return cls_loss

In [26]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    if OHEM:
        loss0 = ohem_loss(outputs[0], y_true[:, 0])
        loss1 = ohem_loss(outputs[1], y_true[:, 1])
        loss2 = ohem_loss(outputs[2], y_true[:, 2])
    else:
        loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
        loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
        loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [27]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [28]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [29]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [30]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [31]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [32]:
np.random.random()

0.25011776135336883

In [33]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from over9000.lookahead import Lookahead
def LookaheadSGD(params, alpha=0.5, k=6, *args, **kwargs):
     sgd = optim.SGD(params, *args, **kwargs)
     return Lookahead(sgd, alpha, k)

In [34]:
def train(args):
    global model

    if args.optim == 'Adam':
        #optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
        optimizer = optim.Adam( [{'params': model.parameters(), 'initial_lr': args.lr }], lr=args.lr, weight_decay=1e-5)
    elif args.optim == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    elif args.optim == 'Over9000':
        optimizer = Over9000(model.parameters(), lr=args.lr)
    else:
        #optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-5)
        optimizer = LookaheadSGD(
            [{'params': model.parameters(), 'initial_lr': args.lr }],
            lr=args.lr, momentum=0.9, weight_decay=1e-5)


    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        #lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.t_max, eta_min=args.min_lr)
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer, args.t_max, eta_min=args.min_lr, last_epoch=args.t_max)
        
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    #if args.lrs == 'plateau':
    #    lr_scheduler.step(best_metrics)
    #else:
    #    lr_scheduler.step()
    train_iter = 0
    
    grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

    for epoch in range(args.num_epochs):

        grid.set_prob(epoch, args.st_epochs)

        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #do_mixup = False #(np.random.random() < 0.4)
            
            #if do_mixup:
            #    img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            
            
            #if do_mixup:
            #    loss = mixup_criterion(outputs, targets)
            #else:
            #    loss = criterion(outputs, targets)
            r = np.random.rand()
            #if args.beta > 0 and r < args.cutmix_prob:
            if r < 0.3:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs = model(img)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            elif r > 0.6: # grid mask
                img = grid(img)
                outputs = model(img)
                loss = criterion(outputs, targets)
            else:
                img, targets = mixup(img, targets)
                outputs = model(img)
                loss = mixup_criterion(outputs, targets)
                #loss = criterion(outputs, targets)
            
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            
            #loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [35]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'model3_se_resnext50_fold4_mixup_cutmix_224_gridmask.pth'
args.predict = False
args.optim = 'Adam'
args.lr = 1e-5
args.lrs = 'plateau'
args.t_max = 15
args.factor = 0.5
args.patience = 10
args.min_lr = 5e-6
args.iter_val = 200
args.num_epochs = 100000
args.batch_size = 768
args.val_batch_size = 1024
args.st_epochs = 1

args.beta = 1.0
args.cutmix_prob = 0.5

In [36]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=4)

(200840, 5)
(200840, 32332)
(160735, 5) (40105, 5)


In [37]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


model file: ./models/se_resnext50_32x4d/model3_se_resnext50_fold4_mixup_cutmix_224_gridmask.pth, exist: True
loading ./models/se_resnext50_32x4d/model3_se_resnext50_fold4_mixup_cutmix_224_gridmask.pth...


In [38]:
train(args)

{'recall': 0.997501, 'recall_grapheme': 0.996235, 'recall_vowel': 0.998718, 'recall_consonant': 0.998817, 'acc_grapheme': 0.996085, 'acc_vowel': 0.998454, 'acc_consonant': 0.998529, 'loss_grapheme': 0.02314, 'loss_vowel': 0.014404, 'loss_consonant': 0.011319}
    0 | 0.000010 | 153600/160735 | 1.7304 | 1.1848 |
val: {'recall': 0.996985, 'recall_grapheme': 0.995763, 'recall_vowel': 0.998175, 'recall_consonant': 0.998239, 'acc_grapheme': 0.995487, 'acc_vowel': 0.99803, 'acc_consonant': 0.998404, 'loss_grapheme': 0.053139, 'loss_vowel': 0.041199, 'loss_consonant': 0.031459}
    1 | 0.000010 | 146688/160735 | 1.9859 | 1.4285 |
val: {'recall': 0.996363, 'recall_grapheme': 0.994541, 'recall_vowel': 0.998401, 'recall_consonant': 0.99797, 'acc_grapheme': 0.994215, 'acc_vowel': 0.998255, 'acc_consonant': 0.99813, 'loss_grapheme': 0.038344, 'loss_vowel': 0.026924, 'loss_consonant': 0.024173}
    2 | 0.000010 | 139776/160735 | 0.0029 | 1.4983 |
val: {'recall': 0.99671, 'recall_grapheme': 0.995177

KeyboardInterrupt: 

In [38]:
train(args)

{'recall': 0.997401, 'recall_grapheme': 0.996331, 'recall_vowel': 0.998438, 'recall_consonant': 0.998503, 'acc_grapheme': 0.995961, 'acc_vowel': 0.998479, 'acc_consonant': 0.998354, 'loss_grapheme': 0.019338, 'loss_vowel': 0.009742, 'loss_consonant': 0.007437}
    0 | 0.000010 | 153600/160735 | 0.0020 | 1.6153 |
val: {'recall': 0.996839, 'recall_grapheme': 0.99563, 'recall_vowel': 0.998025, 'recall_consonant': 0.99807, 'acc_grapheme': 0.995213, 'acc_vowel': 0.998005, 'acc_consonant': 0.99813, 'loss_grapheme': 0.060564, 'loss_vowel': 0.036382, 'loss_consonant': 0.025093}


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    1 | 0.000011 | 146688/160735 | 3.5224 | 1.3788 |
val: {'recall': 0.995218, 'recall_grapheme': 0.993095, 'recall_vowel': 0.997353, 'recall_consonant': 0.997327, 'acc_grapheme': 0.992345, 'acc_vowel': 0.997756, 'acc_consonant': 0.997407, 'loss_grapheme': 0.042741, 'loss_vowel': 0.025682, 'loss_consonant': 0.020981}
    2 | 0.000014 | 139776/160735 | 3.5199 | 1.6067 |
val: {'recall': 0.99583, 'recall_grapheme': 0.993991, 'recall_vowel': 0.997794, 'recall_consonant': 0.997542, 'acc_grapheme': 0.993442, 'acc_vowel': 0.99793, 'acc_consonant': 0.997656, 'loss_grapheme': 0.035975, 'loss_vowel': 0.022077, 'loss_consonant': 0.017602}
    3 | 0.000019 | 132864/160735 | 3.8770 | 1.5255 |
val: {'recall': 0.995595, 'recall_grapheme': 0.993469, 'recall_vowel': 0.997966, 'recall_consonant': 0.997475, 'acc_grapheme': 0.993392, 'acc_vowel': 0.997856, 'acc_consonant': 0.997781, 'loss_grapheme': 0.041311, 'loss_vowel': 0.0252, 'loss_consonant': 0.018565}
    4 | 0.000025 | 125952/160735 | 3.1824 | 1.5

   26 | 0.000019 | 127488/160735 | 0.0035 | 1.5040 |
val: {'recall': 0.997085, 'recall_grapheme': 0.99587, 'recall_vowel': 0.998429, 'recall_consonant': 0.99817, 'acc_grapheme': 0.995761, 'acc_vowel': 0.998429, 'acc_consonant': 0.998479, 'loss_grapheme': 0.023539, 'loss_vowel': 0.014592, 'loss_consonant': 0.011562}
   27 | 0.000014 | 120576/160735 | 2.5166 | 1.3845 |
val: {'recall': 0.995928, 'recall_grapheme': 0.9939, 'recall_vowel': 0.998184, 'recall_consonant': 0.997727, 'acc_grapheme': 0.993741, 'acc_vowel': 0.99798, 'acc_consonant': 0.99818, 'loss_grapheme': 0.032854, 'loss_vowel': 0.020566, 'loss_consonant': 0.015814}
   28 | 0.000011 | 113664/160735 | 0.0062 | 1.4741 |
val: {'recall': 0.99648, 'recall_grapheme': 0.995095, 'recall_vowel': 0.99793, 'recall_consonant': 0.997802, 'acc_grapheme': 0.994814, 'acc_vowel': 0.998304, 'acc_consonant': 0.998205, 'loss_grapheme': 0.030656, 'loss_vowel': 0.019839, 'loss_consonant': 0.016338}
   29 | 0.000010 | 106752/160735 | 4.1728 | 1.5306 

   51 | 0.000050 | 108288/160735 | 1.8061 | 1.5091 |
val: {'recall': 0.997004, 'recall_grapheme': 0.995749, 'recall_vowel': 0.998421, 'recall_consonant': 0.998097, 'acc_grapheme': 0.995861, 'acc_vowel': 0.998529, 'acc_consonant': 0.998379, 'loss_grapheme': 0.032599, 'loss_vowel': 0.026159, 'loss_consonant': 0.021621}
   52 | 0.000041 | 101376/160735 | 0.0140 | 1.1802 |
val: {'recall': 0.996825, 'recall_grapheme': 0.995656, 'recall_vowel': 0.998046, 'recall_consonant': 0.997939, 'acc_grapheme': 0.995487, 'acc_vowel': 0.99828, 'acc_consonant': 0.998329, 'loss_grapheme': 0.026553, 'loss_vowel': 0.01744, 'loss_consonant': 0.01445}
   53 | 0.000033 | 094464/160735 | 0.0026 | 1.6212 |
val: {'recall': 0.996057, 'recall_grapheme': 0.994113, 'recall_vowel': 0.998252, 'recall_consonant': 0.997751, 'acc_grapheme': 0.993791, 'acc_vowel': 0.998055, 'acc_consonant': 0.998105, 'loss_grapheme': 0.042011, 'loss_vowel': 0.029923, 'loss_consonant': 0.024202}
   54 | 0.000025 | 087552/160735 | 4.2028 | 1.

   76 | 0.000085 | 089088/160735 | 2.2495 | 1.2562 |
val: {'recall': 0.996712, 'recall_grapheme': 0.995178, 'recall_vowel': 0.998273, 'recall_consonant': 0.998221, 'acc_grapheme': 0.995013, 'acc_vowel': 0.998155, 'acc_consonant': 0.998329, 'loss_grapheme': 0.038606, 'loss_vowel': 0.029542, 'loss_consonant': 0.02323}
   77 | 0.000077 | 082176/160735 | 0.0048 | 1.4596 |
val: {'recall': 0.996517, 'recall_grapheme': 0.994832, 'recall_vowel': 0.998342, 'recall_consonant': 0.998061, 'acc_grapheme': 0.994564, 'acc_vowel': 0.998304, 'acc_consonant': 0.998255, 'loss_grapheme': 0.030777, 'loss_vowel': 0.021989, 'loss_consonant': 0.018679}
   78 | 0.000069 | 075264/160735 | 3.1716 | 1.2449 |
val: {'recall': 0.997049, 'recall_grapheme': 0.995835, 'recall_vowel': 0.99839, 'recall_consonant': 0.998137, 'acc_grapheme': 0.995636, 'acc_vowel': 0.998429, 'acc_consonant': 0.998504, 'loss_grapheme': 0.032151, 'loss_vowel': 0.025106, 'loss_consonant': 0.021874}
   79 | 0.000060 | 068352/160735 | 2.9126 | 1

KeyboardInterrupt: 

In [79]:
train(args)

{'recall': 0.997401, 'recall_grapheme': 0.996331, 'recall_vowel': 0.998438, 'recall_consonant': 0.998503, 'acc_grapheme': 0.995961, 'acc_vowel': 0.998479, 'acc_consonant': 0.998354, 'loss_grapheme': 0.019338, 'loss_vowel': 0.009742, 'loss_consonant': 0.007437}
    0 | 0.000050 | 153600/160735 | 1.1514 | 1.0147 |
val: {'recall': 0.99651, 'recall_grapheme': 0.994837, 'recall_vowel': 0.99797, 'recall_consonant': 0.998399, 'acc_grapheme': 0.994664, 'acc_vowel': 0.997905, 'acc_consonant': 0.997905, 'loss_grapheme': 0.033601, 'loss_vowel': 0.019487, 'loss_consonant': 0.013147}


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    1 | 0.000049 | 146688/160735 | 1.7823 | 1.1079 |
val: {'recall': 0.99458, 'recall_grapheme': 0.992724, 'recall_vowel': 0.997169, 'recall_consonant': 0.995701, 'acc_grapheme': 0.99227, 'acc_vowel': 0.997507, 'acc_consonant': 0.997357, 'loss_grapheme': 0.05443, 'loss_vowel': 0.035074, 'loss_consonant': 0.024806}
    2 | 0.000047 | 139776/160735 | 1.3825 | 0.9587 |
val: {'recall': 0.996538, 'recall_grapheme': 0.995082, 'recall_vowel': 0.997917, 'recall_consonant': 0.998071, 'acc_grapheme': 0.995287, 'acc_vowel': 0.99803, 'acc_consonant': 0.998105, 'loss_grapheme': 0.026622, 'loss_vowel': 0.015517, 'loss_consonant': 0.010374}
    3 | 0.000043 | 132864/160735 | 0.2587 | 0.9630 |
val: {'recall': 0.993723, 'recall_grapheme': 0.991549, 'recall_vowel': 0.99695, 'recall_consonant': 0.994846, 'acc_grapheme': 0.991772, 'acc_vowel': 0.997133, 'acc_consonant': 0.997058, 'loss_grapheme': 0.044781, 'loss_vowel': 0.020896, 'loss_consonant': 0.016922}
    4 | 0.000038 | 125952/160735 | 2.5432 | 1.12

   26 | 0.000043 | 127488/160735 | 0.0047 | 0.9961 |
val: {'recall': 0.996007, 'recall_grapheme': 0.994401, 'recall_vowel': 0.997091, 'recall_consonant': 0.998135, 'acc_grapheme': 0.993941, 'acc_vowel': 0.997681, 'acc_consonant': 0.997881, 'loss_grapheme': 0.030932, 'loss_vowel': 0.016257, 'loss_consonant': 0.012754}
   27 | 0.000038 | 120576/160735 | 2.9881 | 1.1158 |
val: {'recall': 0.994956, 'recall_grapheme': 0.992688, 'recall_vowel': 0.997026, 'recall_consonant': 0.997421, 'acc_grapheme': 0.992046, 'acc_vowel': 0.997531, 'acc_consonant': 0.997507, 'loss_grapheme': 0.04463, 'loss_vowel': 0.02532, 'loss_consonant': 0.018043}
   28 | 0.000032 | 113664/160735 | 1.9251 | 0.9913 |
val: {'recall': 0.991583, 'recall_grapheme': 0.989058, 'recall_vowel': 0.996288, 'recall_consonant': 0.99193, 'acc_grapheme': 0.98843, 'acc_vowel': 0.996559, 'acc_consonant': 0.996035, 'loss_grapheme': 0.049937, 'loss_vowel': 0.01845, 'loss_consonant': 0.016871}
   29 | 0.000026 | 106752/160735 | 0.0038 | 1.23

KeyboardInterrupt: 

In [37]:
train(args)

{'recall': 0.994807, 'recall_grapheme': 0.993211, 'recall_vowel': 0.997174, 'recall_consonant': 0.995633, 'acc_grapheme': 0.992968, 'acc_vowel': 0.997357, 'acc_consonant': 0.997033, 'loss_grapheme': 0.136084, 'loss_vowel': 0.089044, 'loss_consonant': 0.061338}
    0 | 0.000100 | 153600/160735 | 0.0037 | 1.1513 |
val: {'recall': 0.989714, 'recall_grapheme': 0.984654, 'recall_vowel': 0.994842, 'recall_consonant': 0.994707, 'acc_grapheme': 0.985538, 'acc_vowel': 0.995287, 'acc_consonant': 0.99424, 'loss_grapheme': 0.103949, 'loss_vowel': 0.063651, 'loss_consonant': 0.046677}


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    1 | 0.000098 | 146688/160735 | 1.0486 | 1.0954 |
val: {'recall': 0.992231, 'recall_grapheme': 0.989128, 'recall_vowel': 0.995653, 'recall_consonant': 0.995015, 'acc_grapheme': 0.988854, 'acc_vowel': 0.99616, 'acc_consonant': 0.995537, 'loss_grapheme': 0.10485, 'loss_vowel': 0.072468, 'loss_consonant': 0.047801}
    2 | 0.000093 | 139776/160735 | 1.8618 | 1.0786 |
val: {'recall': 0.992574, 'recall_grapheme': 0.989244, 'recall_vowel': 0.995393, 'recall_consonant': 0.996413, 'acc_grapheme': 0.989602, 'acc_vowel': 0.99606, 'acc_consonant': 0.995886, 'loss_grapheme': 0.117554, 'loss_vowel': 0.077483, 'loss_consonant': 0.056601}
    3 | 0.000086 | 132864/160735 | 1.5228 | 1.0907 |
val: {'recall': 0.993971, 'recall_grapheme': 0.991248, 'recall_vowel': 0.996915, 'recall_consonant': 0.996473, 'acc_grapheme': 0.99045, 'acc_vowel': 0.997008, 'acc_consonant': 0.996584, 'loss_grapheme': 0.080691, 'loss_vowel': 0.054964, 'loss_consonant': 0.03495}
    4 | 0.000075 | 125952/160735 | 0.0120 | 1.05

** saved
   26 | 0.000086 | 127488/160735 | 3.4098 | 1.1860 |
val: {'recall': 0.99461, 'recall_grapheme': 0.991818, 'recall_vowel': 0.997081, 'recall_consonant': 0.997721, 'acc_grapheme': 0.991672, 'acc_vowel': 0.996808, 'acc_consonant': 0.996783, 'loss_grapheme': 0.064706, 'loss_vowel': 0.048355, 'loss_consonant': 0.035835}
   27 | 0.000075 | 120576/160735 | 0.0073 | 1.0463 |
val: {'recall': 0.994773, 'recall_grapheme': 0.992577, 'recall_vowel': 0.997597, 'recall_consonant': 0.996342, 'acc_grapheme': 0.99257, 'acc_vowel': 0.997606, 'acc_consonant': 0.997182, 'loss_grapheme': 0.043618, 'loss_vowel': 0.028452, 'loss_consonant': 0.022686}
   28 | 0.000063 | 113664/160735 | 2.6333 | 1.1025 |
val: {'recall': 0.993794, 'recall_grapheme': 0.991102, 'recall_vowel': 0.996929, 'recall_consonant': 0.996045, 'acc_grapheme': 0.9905, 'acc_vowel': 0.997083, 'acc_consonant': 0.996584, 'loss_grapheme': 0.074834, 'loss_vowel': 0.060056, 'loss_consonant': 0.045007}
   29 | 0.000051 | 106752/160735 | 0.5

   51 | 0.000063 | 108288/160735 | 1.5406 | 0.9697 |
val: {'recall': 0.993109, 'recall_grapheme': 0.990217, 'recall_vowel': 0.996522, 'recall_consonant': 0.995479, 'acc_grapheme': 0.988929, 'acc_vowel': 0.996833, 'acc_consonant': 0.995961, 'loss_grapheme': 0.056458, 'loss_vowel': 0.034883, 'loss_consonant': 0.031278}
   52 | 0.000051 | 101376/160735 | 0.0033 | 1.1372 |
val: {'recall': 0.995313, 'recall_grapheme': 0.99295, 'recall_vowel': 0.997803, 'recall_consonant': 0.997548, 'acc_grapheme': 0.992495, 'acc_vowel': 0.997756, 'acc_consonant': 0.997182, 'loss_grapheme': 0.040742, 'loss_vowel': 0.026209, 'loss_consonant': 0.020372}
   53 | 0.000038 | 094464/160735 | 0.0121 | 0.8790 |
val: {'recall': 0.996658, 'recall_grapheme': 0.994888, 'recall_vowel': 0.998465, 'recall_consonant': 0.998392, 'acc_grapheme': 0.994888, 'acc_vowel': 0.99828, 'acc_consonant': 0.99793, 'loss_grapheme': 0.031662, 'loss_vowel': 0.017433, 'loss_consonant': 0.013885}
** saved
   54 | 0.000026 | 087552/160735 | 0.

...

  101 | 0.000015 | 069888/160735 | 1.6693 | 0.9381 |
val: {'recall': 0.996133, 'recall_grapheme': 0.994301, 'recall_vowel': 0.997649, 'recall_consonant': 0.99828, 'acc_grapheme': 0.993667, 'acc_vowel': 0.997831, 'acc_consonant': 0.997955, 'loss_grapheme': 0.037403, 'loss_vowel': 0.02273, 'loss_consonant': 0.016761}
  102 | 0.000008 | 062976/160735 | 0.0031 | 0.9741 |
val: {'recall': 0.997018, 'recall_grapheme': 0.995462, 'recall_vowel': 0.998495, 'recall_consonant': 0.998655, 'acc_grapheme': 0.995088, 'acc_vowel': 0.998354, 'acc_consonant': 0.998404, 'loss_grapheme': 0.02418, 'loss_vowel': 0.011886, 'loss_consonant': 0.008562}
  103 | 0.000003 | 056064/160735 | 1.6289 | 0.8747 |
val: {'recall': 0.996627, 'recall_grapheme': 0.995, 'recall_vowel': 0.998122, 'recall_consonant': 0.998386, 'acc_grapheme': 0.99429, 'acc_vowel': 0.998055, 'acc_consonant': 0.99808, 'loss_grapheme': 0.031662, 'loss_vowel': 0.018117, 'loss_consonant': 0.013088}
  104 | 0.000001 | 049152/160735 | 0.0041 | 0.9578 

  126 | 0.000003 | 050688/160735 | 2.1043 | 0.9254 |
val: {'recall': 0.996198, 'recall_grapheme': 0.994444, 'recall_vowel': 0.998057, 'recall_consonant': 0.997847, 'acc_grapheme': 0.994315, 'acc_vowel': 0.997955, 'acc_consonant': 0.99808, 'loss_grapheme': 0.032837, 'loss_vowel': 0.021154, 'loss_consonant': 0.015282}
  127 | 0.000001 | 043776/160735 | 0.0213 | 1.0302 |
val: {'recall': 0.995947, 'recall_grapheme': 0.994032, 'recall_vowel': 0.997894, 'recall_consonant': 0.997829, 'acc_grapheme': 0.993567, 'acc_vowel': 0.997905, 'acc_consonant': 0.997905, 'loss_grapheme': 0.030067, 'loss_vowel': 0.014481, 'loss_consonant': 0.011062}
  128 | 0.000003 | 036864/160735 | 0.4714 | 1.1853 |
val: {'recall': 0.995707, 'recall_grapheme': 0.993753, 'recall_vowel': 0.99765, 'recall_consonant': 0.997672, 'acc_grapheme': 0.993118, 'acc_vowel': 0.997781, 'acc_consonant': 0.997581, 'loss_grapheme': 0.048194, 'loss_vowel': 0.033156, 'loss_consonant': 0.02459}
  129 | 0.000008 | 029952/160735 | 1.4470 | 0.

  151 | 0.000003 | 031488/160735 | 2.0051 | 1.0539 |
val: {'recall': 0.99659, 'recall_grapheme': 0.99497, 'recall_vowel': 0.997655, 'recall_consonant': 0.998764, 'acc_grapheme': 0.994564, 'acc_vowel': 0.998055, 'acc_consonant': 0.998205, 'loss_grapheme': 0.029236, 'loss_vowel': 0.017448, 'loss_consonant': 0.011928}
  152 | 0.000008 | 024576/160735 | 1.1848 | 0.8531 |
val: {'recall': 0.997071, 'recall_grapheme': 0.995564, 'recall_vowel': 0.998247, 'recall_consonant': 0.99891, 'acc_grapheme': 0.995362, 'acc_vowel': 0.998329, 'acc_consonant': 0.998479, 'loss_grapheme': 0.023826, 'loss_vowel': 0.013686, 'loss_consonant': 0.009157}
  153 | 0.000015 | 017664/160735 | 2.1579 | 0.9579 |
val: {'recall': 0.996451, 'recall_grapheme': 0.994517, 'recall_vowel': 0.998093, 'recall_consonant': 0.998676, 'acc_grapheme': 0.994265, 'acc_vowel': 0.99813, 'acc_consonant': 0.99808, 'loss_grapheme': 0.037137, 'loss_vowel': 0.022186, 'loss_consonant': 0.015718}
  154 | 0.000026 | 010752/160735 | 2.4124 | 1.16

  176 | 0.000015 | 012288/160735 | 2.4402 | 0.9590 |
val: {'recall': 0.995664, 'recall_grapheme': 0.993392, 'recall_vowel': 0.997608, 'recall_consonant': 0.998265, 'acc_grapheme': 0.993218, 'acc_vowel': 0.997756, 'acc_consonant': 0.997731, 'loss_grapheme': 0.032383, 'loss_vowel': 0.015922, 'loss_consonant': 0.012419}
  177 | 0.000026 | 005376/160735 | 1.4361 | 1.1820 |
val: {'recall': 0.99608, 'recall_grapheme': 0.9943, 'recall_vowel': 0.997567, 'recall_consonant': 0.998153, 'acc_grapheme': 0.993916, 'acc_vowel': 0.997806, 'acc_consonant': 0.997856, 'loss_grapheme': 0.028746, 'loss_vowel': 0.01329, 'loss_consonant': 0.010584}
  177 | 0.000038 | 158976/160735 | 0.0045 | 0.9369 |
val: {'recall': 0.996069, 'recall_grapheme': 0.994068, 'recall_vowel': 0.997655, 'recall_consonant': 0.998485, 'acc_grapheme': 0.993667, 'acc_vowel': 0.997856, 'acc_consonant': 0.99793, 'loss_grapheme': 0.033645, 'loss_vowel': 0.018588, 'loss_consonant': 0.013091}
  178 | 0.000050 | 152064/160735 | 0.0043 | 1.01

KeyboardInterrupt: 

In [ ]:
train(args)

{'recall': 0.968796, 'recall_grapheme': 0.954354, 'recall_vowel': 0.981436, 'recall_consonant': 0.985042, 'acc_grapheme': 0.950954, 'acc_vowel': 0.981798, 'acc_consonant': 0.982496, 'loss_grapheme': 0.378759, 'loss_vowel': 0.200741, 'loss_consonant': 0.14877}
    0 | 0.000100 | 153600/160735 | 2.7201 | 2.1176 |
val: {'recall': 0.987331, 'recall_grapheme': 0.98181, 'recall_vowel': 0.994441, 'recall_consonant': 0.991263, 'acc_grapheme': 0.982521, 'acc_vowel': 0.994714, 'acc_consonant': 0.99424, 'loss_grapheme': 0.20238, 'loss_vowel': 0.171463, 'loss_consonant': 0.109966}
** saved


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    1 | 0.000098 | 146688/160735 | 1.9706 | 2.0632 |
val: {'recall': 0.98721, 'recall_grapheme': 0.981044, 'recall_vowel': 0.993634, 'recall_consonant': 0.993119, 'acc_grapheme': 0.98297, 'acc_vowel': 0.994564, 'acc_consonant': 0.994789, 'loss_grapheme': 0.193056, 'loss_vowel': 0.165949, 'loss_consonant': 0.103149}
    2 | 0.000093 | 139776/160735 | 1.5632 | 1.9585 |
val: {'recall': 0.988972, 'recall_grapheme': 0.983604, 'recall_vowel': 0.993653, 'recall_consonant': 0.995028, 'acc_grapheme': 0.983992, 'acc_vowel': 0.994689, 'acc_consonant': 0.994639, 'loss_grapheme': 0.184705, 'loss_vowel': 0.15318, 'loss_consonant': 0.105877}
** saved
    3 | 0.000086 | 132864/160735 | 1.7528 | 2.1131 |
val: {'recall': 0.989968, 'recall_grapheme': 0.985642, 'recall_vowel': 0.994857, 'recall_consonant': 0.993729, 'acc_grapheme': 0.98469, 'acc_vowel': 0.995437, 'acc_consonant': 0.995138, 'loss_grapheme': 0.207736, 'loss_vowel': 0.187592, 'loss_consonant': 0.11266}
** saved
    4 | 0.000075 | 125952/1607

   26 | 0.000086 | 127488/160735 | 0.6730 | 1.8537 |
val: {'recall': 0.986989, 'recall_grapheme': 0.980761, 'recall_vowel': 0.993584, 'recall_consonant': 0.992849, 'acc_grapheme': 0.983369, 'acc_vowel': 0.994714, 'acc_consonant': 0.994514, 'loss_grapheme': 0.165355, 'loss_vowel': 0.096894, 'loss_consonant': 0.069883}
   27 | 0.000075 | 120576/160735 | 1.5488 | 1.8665 |
val: {'recall': 0.989669, 'recall_grapheme': 0.985103, 'recall_vowel': 0.994121, 'recall_consonant': 0.994349, 'acc_grapheme': 0.984241, 'acc_vowel': 0.995437, 'acc_consonant': 0.994664, 'loss_grapheme': 0.199886, 'loss_vowel': 0.181033, 'loss_consonant': 0.106551}
   28 | 0.000063 | 113664/160735 | 1.7374 | 1.8715 |
val: {'recall': 0.991073, 'recall_grapheme': 0.986988, 'recall_vowel': 0.99497, 'recall_consonant': 0.995346, 'acc_grapheme': 0.987408, 'acc_vowel': 0.995562, 'acc_consonant': 0.995312, 'loss_grapheme': 0.119674, 'loss_vowel': 0.082174, 'loss_consonant': 0.058381}
   29 | 0.000051 | 106752/160735 | 1.5602 | 

   51 | 0.000063 | 108288/160735 | 0.8534 | 1.9240 |
val: {'recall': 0.992213, 'recall_grapheme': 0.988386, 'recall_vowel': 0.996299, 'recall_consonant': 0.995783, 'acc_grapheme': 0.988131, 'acc_vowel': 0.995986, 'acc_consonant': 0.995886, 'loss_grapheme': 0.172872, 'loss_vowel': 0.14041, 'loss_consonant': 0.089729}
   52 | 0.000051 | 101376/160735 | 1.8692 | 1.8458 |
val: {'recall': 0.992772, 'recall_grapheme': 0.989642, 'recall_vowel': 0.996197, 'recall_consonant': 0.995609, 'acc_grapheme': 0.989453, 'acc_vowel': 0.996335, 'acc_consonant': 0.995886, 'loss_grapheme': 0.135381, 'loss_vowel': 0.090325, 'loss_consonant': 0.06973}
** saved
   53 | 0.000038 | 094464/160735 | 0.6151 | 1.8515 |
val: {'recall': 0.992988, 'recall_grapheme': 0.989467, 'recall_vowel': 0.995835, 'recall_consonant': 0.997184, 'acc_grapheme': 0.988979, 'acc_vowel': 0.996509, 'acc_consonant': 0.99631, 'loss_grapheme': 0.170543, 'loss_vowel': 0.143205, 'loss_consonant': 0.086028}
** saved
   54 | 0.000026 | 087552/16

...

** saved
  101 | 0.000015 | 069888/160735 | 2.0242 | 1.6856 |
val: {'recall': 0.992789, 'recall_grapheme': 0.990106, 'recall_vowel': 0.996376, 'recall_consonant': 0.994569, 'acc_grapheme': 0.990874, 'acc_vowel': 0.996883, 'acc_consonant': 0.996384, 'loss_grapheme': 0.167718, 'loss_vowel': 0.110026, 'loss_consonant': 0.082676}
  102 | 0.000008 | 062976/160735 | 2.4685 | 1.7904 |
val: {'recall': 0.993504, 'recall_grapheme': 0.990287, 'recall_vowel': 0.996599, 'recall_consonant': 0.996845, 'acc_grapheme': 0.989752, 'acc_vowel': 0.997083, 'acc_consonant': 0.996335, 'loss_grapheme': 0.170766, 'loss_vowel': 0.149114, 'loss_consonant': 0.089225}
  103 | 0.000003 | 056064/160735 | 2.1733 | 1.8125 |
val: {'recall': 0.993311, 'recall_grapheme': 0.989895, 'recall_vowel': 0.996625, 'recall_consonant': 0.996829, 'acc_grapheme': 0.989702, 'acc_vowel': 0.997083, 'acc_consonant': 0.996484, 'loss_grapheme': 0.186024, 'loss_vowel': 0.164717, 'loss_consonant': 0.096067}
  104 | 0.000001 | 049152/160735 |

  126 | 0.000003 | 050688/160735 | 1.1572 | 1.8722 |
val: {'recall': 0.994112, 'recall_grapheme': 0.991223, 'recall_vowel': 0.996817, 'recall_consonant': 0.997184, 'acc_grapheme': 0.991547, 'acc_vowel': 0.997157, 'acc_consonant': 0.996759, 'loss_grapheme': 0.179152, 'loss_vowel': 0.144877, 'loss_consonant': 0.087624}
  127 | 0.000001 | 043776/160735 | 2.4575 | 1.7891 |
val: {'recall': 0.993426, 'recall_grapheme': 0.989912, 'recall_vowel': 0.996859, 'recall_consonant': 0.997023, 'acc_grapheme': 0.990026, 'acc_vowel': 0.997033, 'acc_consonant': 0.99616, 'loss_grapheme': 0.176595, 'loss_vowel': 0.157275, 'loss_consonant': 0.090055}
  128 | 0.000003 | 036864/160735 | 1.3953 | 1.7547 |
val: {'recall': 0.993973, 'recall_grapheme': 0.990907, 'recall_vowel': 0.996913, 'recall_consonant': 0.997167, 'acc_grapheme': 0.990699, 'acc_vowel': 0.997157, 'acc_consonant': 0.996434, 'loss_grapheme': 0.160871, 'loss_vowel': 0.133616, 'loss_consonant': 0.079128}
  129 | 0.000008 | 029952/160735 | 3.5278 | 

** saved
  151 | 0.000003 | 031488/160735 | 1.4664 | 1.8525 |
val: {'recall': 0.993524, 'recall_grapheme': 0.990969, 'recall_vowel': 0.996435, 'recall_consonant': 0.995726, 'acc_grapheme': 0.990051, 'acc_vowel': 0.996883, 'acc_consonant': 0.996434, 'loss_grapheme': 0.169892, 'loss_vowel': 0.152663, 'loss_consonant': 0.094209}
  152 | 0.000008 | 024576/160735 | 0.8803 | 1.5736 |
val: {'recall': 0.994322, 'recall_grapheme': 0.992729, 'recall_vowel': 0.996545, 'recall_consonant': 0.995284, 'acc_grapheme': 0.992669, 'acc_vowel': 0.996958, 'acc_consonant': 0.996933, 'loss_grapheme': 0.121052, 'loss_vowel': 0.078456, 'loss_consonant': 0.056436}
  153 | 0.000015 | 017664/160735 | 1.7538 | 1.7494 |
val: {'recall': 0.993763, 'recall_grapheme': 0.990911, 'recall_vowel': 0.996625, 'recall_consonant': 0.996606, 'acc_grapheme': 0.9903, 'acc_vowel': 0.997033, 'acc_consonant': 0.996833, 'loss_grapheme': 0.163015, 'loss_vowel': 0.146479, 'loss_consonant': 0.086465}
  154 | 0.000026 | 010752/160735 | 2

** saved
  176 | 0.000015 | 012288/160735 | 1.9095 | 1.9380 |
val: {'recall': 0.992947, 'recall_grapheme': 0.989441, 'recall_vowel': 0.995992, 'recall_consonant': 0.996914, 'acc_grapheme': 0.988854, 'acc_vowel': 0.996634, 'acc_consonant': 0.99611, 'loss_grapheme': 0.148052, 'loss_vowel': 0.122292, 'loss_consonant': 0.080409}
  177 | 0.000026 | 005376/160735 | 2.1672 | 1.8442 |
val: {'recall': 0.993734, 'recall_grapheme': 0.990806, 'recall_vowel': 0.996438, 'recall_consonant': 0.996885, 'acc_grapheme': 0.990176, 'acc_vowel': 0.996908, 'acc_consonant': 0.996534, 'loss_grapheme': 0.152286, 'loss_vowel': 0.113841, 'loss_consonant': 0.07727}
  177 | 0.000038 | 158976/160735 | 2.0699 | 1.8261 |
val: {'recall': 0.992541, 'recall_grapheme': 0.988906, 'recall_vowel': 0.99569, 'recall_consonant': 0.99666, 'acc_grapheme': 0.988181, 'acc_vowel': 0.996459, 'acc_consonant': 0.995761, 'loss_grapheme': 0.155896, 'loss_vowel': 0.130231, 'loss_consonant': 0.088185}
  178 | 0.000050 | 152064/160735 | 1.8

  200 | 0.000038 | 153600/160735 | 0.7268 | 1.6830 |
val: {'recall': 0.993738, 'recall_grapheme': 0.991404, 'recall_vowel': 0.996603, 'recall_consonant': 0.995542, 'acc_grapheme': 0.991248, 'acc_vowel': 0.996983, 'acc_consonant': 0.996858, 'loss_grapheme': 0.096999, 'loss_vowel': 0.065398, 'loss_consonant': 0.048689}
  201 | 0.000051 | 146688/160735 | 1.5096 | 1.7420 |
val: {'recall': 0.992967, 'recall_grapheme': 0.989507, 'recall_vowel': 0.99619, 'recall_consonant': 0.996662, 'acc_grapheme': 0.988854, 'acc_vowel': 0.996709, 'acc_consonant': 0.995936, 'loss_grapheme': 0.095177, 'loss_vowel': 0.064534, 'loss_consonant': 0.051418}
  202 | 0.000063 | 139776/160735 | 1.7080 | 1.7172 |
val: {'recall': 0.992058, 'recall_grapheme': 0.98805, 'recall_vowel': 0.996103, 'recall_consonant': 0.996028, 'acc_grapheme': 0.987508, 'acc_vowel': 0.996409, 'acc_consonant': 0.995661, 'loss_grapheme': 0.097064, 'loss_vowel': 0.073032, 'loss_consonant': 0.060751}
  203 | 0.000075 | 132864/160735 | 0.9834 | 1

In [36]:
train(args)

{'recall': 0.956802, 'recall_grapheme': 0.932222, 'recall_vowel': 0.980891, 'recall_consonant': 0.981874, 'acc_grapheme': 0.930009, 'acc_vowel': 0.980875, 'acc_consonant': 0.978132, 'loss_grapheme': 0.312673, 'loss_vowel': 0.164236, 'loss_consonant': 0.135241}
    1 | 0.000100 | 045056/160735 | 2.5336 | 2.7666 |
val: {'recall': 0.974535, 'recall_grapheme': 0.960737, 'recall_vowel': 0.986709, 'recall_consonant': 0.989957, 'acc_grapheme': 0.958883, 'acc_vowel': 0.988231, 'acc_consonant': 0.983842, 'loss_grapheme': 0.434106, 'loss_vowel': 0.267702, 'loss_consonant': 0.198019}
** saved


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    2 | 0.000098 | 090112/160735 | 2.6231 | 2.9613 |
val: {'recall': 0.97632, 'recall_grapheme': 0.965324, 'recall_vowel': 0.988569, 'recall_consonant': 0.986062, 'acc_grapheme': 0.962698, 'acc_vowel': 0.987109, 'acc_consonant': 0.986859, 'loss_grapheme': 0.42304, 'loss_vowel': 0.278218, 'loss_consonant': 0.163295}
** saved
    3 | 0.000093 | 135168/160735 | 3.3173 | 3.0019 |
val: {'recall': 0.977042, 'recall_grapheme': 0.965013, 'recall_vowel': 0.988055, 'recall_consonant': 0.990086, 'acc_grapheme': 0.963546, 'acc_vowel': 0.988281, 'acc_consonant': 0.98676, 'loss_grapheme': 0.365671, 'loss_vowel': 0.220216, 'loss_consonant': 0.152609}
** saved
    5 | 0.000086 | 020480/160735 | 1.6969 | 2.7959 |
val: {'recall': 0.976646, 'recall_grapheme': 0.964318, 'recall_vowel': 0.988691, 'recall_consonant': 0.989257, 'acc_grapheme': 0.963695, 'acc_vowel': 0.988555, 'acc_consonant': 0.988031, 'loss_grapheme': 0.375888, 'loss_vowel': 0.208472, 'loss_consonant': 0.133503}
    6 | 0.000075 | 065536/16

   35 | 0.000086 | 143360/160735 | 4.0223 | 2.2561 |
val: {'recall': 0.981691, 'recall_grapheme': 0.97226, 'recall_vowel': 0.990182, 'recall_consonant': 0.992062, 'acc_grapheme': 0.971375, 'acc_vowel': 0.991024, 'acc_consonant': 0.989951, 'loss_grapheme': 0.297572, 'loss_vowel': 0.215084, 'loss_consonant': 0.139041}
** saved
   37 | 0.000075 | 028672/160735 | 1.6687 | 2.7269 |
val: {'recall': 0.982121, 'recall_grapheme': 0.97351, 'recall_vowel': 0.990875, 'recall_consonant': 0.990588, 'acc_grapheme': 0.972946, 'acc_vowel': 0.991697, 'acc_consonant': 0.99045, 'loss_grapheme': 0.248974, 'loss_vowel': 0.181482, 'loss_consonant': 0.118749}
** saved
   38 | 0.000063 | 073728/160735 | 2.4216 | 2.6370 |
val: {'recall': 0.982752, 'recall_grapheme': 0.974229, 'recall_vowel': 0.990714, 'recall_consonant': 0.991838, 'acc_grapheme': 0.974243, 'acc_vowel': 0.991622, 'acc_consonant': 0.990425, 'loss_grapheme': 0.285626, 'loss_vowel': 0.202272, 'loss_consonant': 0.132067}
** saved
   39 | 0.000051 | 

** saved
   69 | 0.000063 | 036864/160735 | 3.2751 | 2.1040 |
val: {'recall': 0.984252, 'recall_grapheme': 0.976325, 'recall_vowel': 0.992133, 'recall_consonant': 0.992225, 'acc_grapheme': 0.975689, 'acc_vowel': 0.992669, 'acc_consonant': 0.991273, 'loss_grapheme': 0.22588, 'loss_vowel': 0.163569, 'loss_consonant': 0.108342}
   70 | 0.000051 | 081920/160735 | 3.8027 | 2.0982 |
val: {'recall': 0.985088, 'recall_grapheme': 0.977767, 'recall_vowel': 0.992182, 'recall_consonant': 0.992635, 'acc_grapheme': 0.976686, 'acc_vowel': 0.993068, 'acc_consonant': 0.991223, 'loss_grapheme': 0.272577, 'loss_vowel': 0.222287, 'loss_consonant': 0.13877}
** saved
   71 | 0.000038 | 126976/160735 | 3.9181 | 2.1338 |
val: {'recall': 0.984895, 'recall_grapheme': 0.977641, 'recall_vowel': 0.992039, 'recall_consonant': 0.992258, 'acc_grapheme': 0.97726, 'acc_vowel': 0.992395, 'acc_consonant': 0.991447, 'loss_grapheme': 0.236995, 'loss_vowel': 0.18876, 'loss_consonant': 0.1257}
   73 | 0.000026 | 012288/16073

...

  202 | 0.000003 | 090112/160735 | 2.7399 | 2.1298 |
val: {'recall': 0.987244, 'recall_grapheme': 0.980398, 'recall_vowel': 0.993838, 'recall_consonant': 0.99434, 'acc_grapheme': 0.980726, 'acc_vowel': 0.994115, 'acc_consonant': 0.993442, 'loss_grapheme': 0.225589, 'loss_vowel': 0.195456, 'loss_consonant': 0.130961}
  203 | 0.000008 | 135168/160735 | 1.6156 | 1.9866 |
val: {'recall': 0.987448, 'recall_grapheme': 0.981253, 'recall_vowel': 0.99377, 'recall_consonant': 0.993516, 'acc_grapheme': 0.981349, 'acc_vowel': 0.994514, 'acc_consonant': 0.993492, 'loss_grapheme': 0.164991, 'loss_vowel': 0.130408, 'loss_consonant': 0.094289}
  205 | 0.000015 | 020480/160735 | 0.7701 | 1.7087 |
val: {'recall': 0.987128, 'recall_grapheme': 0.981175, 'recall_vowel': 0.993591, 'recall_consonant': 0.992572, 'acc_grapheme': 0.981299, 'acc_vowel': 0.99439, 'acc_consonant': 0.993542, 'loss_grapheme': 0.169065, 'loss_vowel': 0.137996, 'loss_consonant': 0.093789}
  206 | 0.000026 | 065536/160735 | 2.3908 | 1.

  235 | 0.000015 | 143360/160735 | 0.8247 | 1.9545 |
val: {'recall': 0.988526, 'recall_grapheme': 0.98271, 'recall_vowel': 0.994058, 'recall_consonant': 0.994623, 'acc_grapheme': 0.983294, 'acc_vowel': 0.994938, 'acc_consonant': 0.993941, 'loss_grapheme': 0.180107, 'loss_vowel': 0.155644, 'loss_consonant': 0.104287}
  237 | 0.000026 | 028672/160735 | 1.4535 | 1.9321 |
val: {'recall': 0.988865, 'recall_grapheme': 0.983206, 'recall_vowel': 0.994096, 'recall_consonant': 0.99495, 'acc_grapheme': 0.983045, 'acc_vowel': 0.994863, 'acc_consonant': 0.99419, 'loss_grapheme': 0.166113, 'loss_vowel': 0.134965, 'loss_consonant': 0.091629}
  238 | 0.000038 | 073728/160735 | 0.6702 | 1.9239 |
val: {'recall': 0.988107, 'recall_grapheme': 0.982294, 'recall_vowel': 0.993691, 'recall_consonant': 0.994148, 'acc_grapheme': 0.981873, 'acc_vowel': 0.994639, 'acc_consonant': 0.993891, 'loss_grapheme': 0.172843, 'loss_vowel': 0.13561, 'loss_consonant': 0.093455}
  239 | 0.000050 | 118784/160735 | 1.4516 | 2.0

  269 | 0.000038 | 036864/160735 | 2.5354 | 2.1139 |
val: {'recall': 0.989331, 'recall_grapheme': 0.983721, 'recall_vowel': 0.99475, 'recall_consonant': 0.995131, 'acc_grapheme': 0.984366, 'acc_vowel': 0.995088, 'acc_consonant': 0.99434, 'loss_grapheme': 0.16986, 'loss_vowel': 0.143902, 'loss_consonant': 0.098194}
  270 | 0.000051 | 081920/160735 | 2.0034 | 1.9885 |
val: {'recall': 0.989417, 'recall_grapheme': 0.98374, 'recall_vowel': 0.99474, 'recall_consonant': 0.995449, 'acc_grapheme': 0.983394, 'acc_vowel': 0.995013, 'acc_consonant': 0.994041, 'loss_grapheme': 0.229629, 'loss_vowel': 0.20222, 'loss_consonant': 0.137522}
  271 | 0.000063 | 126976/160735 | 1.8902 | 1.9979 |
val: {'recall': 0.988657, 'recall_grapheme': 0.982472, 'recall_vowel': 0.9945, 'recall_consonant': 0.995182, 'acc_grapheme': 0.982621, 'acc_vowel': 0.994938, 'acc_consonant': 0.994514, 'loss_grapheme': 0.163545, 'loss_vowel': 0.122768, 'loss_consonant': 0.082806}
  273 | 0.000075 | 012288/160735 | 3.7529 | 2.0472 

  302 | 0.000063 | 090112/160735 | 1.2726 | 1.7738 |
val: {'recall': 0.988702, 'recall_grapheme': 0.983718, 'recall_vowel': 0.994518, 'recall_consonant': 0.992856, 'acc_grapheme': 0.983045, 'acc_vowel': 0.995262, 'acc_consonant': 0.994789, 'loss_grapheme': 0.183507, 'loss_vowel': 0.144291, 'loss_consonant': 0.097737}
  303 | 0.000075 | 135168/160735 | 1.4575 | 2.0059 |
val: {'recall': 0.988553, 'recall_grapheme': 0.983333, 'recall_vowel': 0.993628, 'recall_consonant': 0.993917, 'acc_grapheme': 0.982521, 'acc_vowel': 0.994764, 'acc_consonant': 0.993891, 'loss_grapheme': 0.184154, 'loss_vowel': 0.147146, 'loss_consonant': 0.100828}
  305 | 0.000086 | 020480/160735 | 3.4163 | 2.2080 |
val: {'recall': 0.987715, 'recall_grapheme': 0.9821, 'recall_vowel': 0.993432, 'recall_consonant': 0.993228, 'acc_grapheme': 0.981947, 'acc_vowel': 0.994215, 'acc_consonant': 0.994091, 'loss_grapheme': 0.221949, 'loss_vowel': 0.172567, 'loss_consonant': 0.117176}
  306 | 0.000093 | 065536/160735 | 0.7306 | 1

KeyboardInterrupt: 

In [40]:
train(args)

{'recall': 0.989245, 'recall_grapheme': 0.984651, 'recall_vowel': 0.99481, 'recall_consonant': 0.992869, 'acc_grapheme': 0.98365, 'acc_vowel': 0.995105, 'acc_consonant': 0.994086, 'loss_grapheme': 0.195219, 'loss_vowel': 0.131839, 'loss_consonant': 0.091149}
    1 | 0.000100 | 045056/160596 | 2.4472 | 1.9134 |
val: {'recall': 0.986924, 'recall_grapheme': 0.98111, 'recall_vowel': 0.993723, 'recall_consonant': 0.991754, 'acc_grapheme': 0.981389, 'acc_vowel': 0.994235, 'acc_consonant': 0.992893, 'loss_grapheme': 0.238637, 'loss_vowel': 0.157462, 'loss_consonant': 0.11222}
    2 | 0.000098 | 090112/160596 | 1.6519 | 1.8694 |
val: {'recall': 0.987757, 'recall_grapheme': 0.981959, 'recall_vowel': 0.993961, 'recall_consonant': 0.993149, 'acc_grapheme': 0.982333, 'acc_vowel': 0.994633, 'acc_consonant': 0.993664, 'loss_grapheme': 0.210871, 'loss_vowel': 0.128744, 'loss_consonant': 0.094444}
    3 | 0.000093 | 135168/160596 | 1.8124 | 1.9584 |
val: {'recall': 0.98804, 'recall_grapheme': 0.981478

   33 | 0.000098 | 053248/160596 | 2.5160 | 2.1916 |
val: {'recall': 0.988107, 'recall_grapheme': 0.982958, 'recall_vowel': 0.994411, 'recall_consonant': 0.992102, 'acc_grapheme': 0.981712, 'acc_vowel': 0.994956, 'acc_consonant': 0.993192, 'loss_grapheme': 0.222498, 'loss_vowel': 0.153475, 'loss_consonant': 0.101955}
   34 | 0.000093 | 098304/160596 | 2.6944 | 1.8929 |
val: {'recall': 0.988882, 'recall_grapheme': 0.983976, 'recall_vowel': 0.994047, 'recall_consonant': 0.993529, 'acc_grapheme': 0.984097, 'acc_vowel': 0.99508, 'acc_consonant': 0.993589, 'loss_grapheme': 0.220375, 'loss_vowel': 0.174837, 'loss_consonant': 0.108714}
   35 | 0.000086 | 143360/160596 | 2.7807 | 1.8129 |
val: {'recall': 0.987663, 'recall_grapheme': 0.98169, 'recall_vowel': 0.994333, 'recall_consonant': 0.992938, 'acc_grapheme': 0.982382, 'acc_vowel': 0.995155, 'acc_consonant': 0.993291, 'loss_grapheme': 0.255152, 'loss_vowel': 0.201604, 'loss_consonant': 0.132399}
   37 | 0.000075 | 028672/160596 | 2.4051 | 1

   66 | 0.000086 | 106496/160596 | 2.5894 | 1.9210 |
val: {'recall': 0.988359, 'recall_grapheme': 0.983217, 'recall_vowel': 0.994168, 'recall_consonant': 0.992832, 'acc_grapheme': 0.982283, 'acc_vowel': 0.995204, 'acc_consonant': 0.993713, 'loss_grapheme': 0.207433, 'loss_vowel': 0.132327, 'loss_consonant': 0.092846}
   67 | 0.000075 | 151552/160596 | 1.2263 | 1.7428 |
val: {'recall': 0.988859, 'recall_grapheme': 0.98391, 'recall_vowel': 0.994468, 'recall_consonant': 0.993148, 'acc_grapheme': 0.98529, 'acc_vowel': 0.995254, 'acc_consonant': 0.993664, 'loss_grapheme': 0.192616, 'loss_vowel': 0.104833, 'loss_consonant': 0.083637}
   69 | 0.000063 | 036864/160596 | 1.7224 | 1.8761 |
val: {'recall': 0.989624, 'recall_grapheme': 0.985094, 'recall_vowel': 0.994841, 'recall_consonant': 0.993467, 'acc_grapheme': 0.985091, 'acc_vowel': 0.995403, 'acc_consonant': 0.99431, 'loss_grapheme': 0.207735, 'loss_vowel': 0.136641, 'loss_consonant': 0.095235}
   70 | 0.000051 | 081920/160596 | 2.4627 | 1.

KeyboardInterrupt: 

In [36]:
train(args)

{'recall': 0.984545, 'recall_grapheme': 0.977655, 'recall_vowel': 0.991144, 'recall_consonant': 0.991725, 'acc_grapheme': 0.976518, 'acc_vowel': 0.993067, 'acc_consonant': 0.991204, 'loss_grapheme': 0.173279, 'loss_vowel': 0.118327, 'loss_consonant': 0.087629}
    1 | 0.000040 | 045056/160596 | 1.1521 | 2.3504 |
val: {'recall': 0.98306, 'recall_grapheme': 0.974674, 'recall_vowel': 0.990755, 'recall_consonant': 0.992138, 'acc_grapheme': 0.975052, 'acc_vowel': 0.992744, 'acc_consonant': 0.991104, 'loss_grapheme': 0.199909, 'loss_vowel': 0.159334, 'loss_consonant': 0.112617}


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    2 | 0.000039 | 090112/160596 | 3.2593 | 2.0820 |
val: {'recall': 0.983107, 'recall_grapheme': 0.976135, 'recall_vowel': 0.990384, 'recall_consonant': 0.989775, 'acc_grapheme': 0.974903, 'acc_vowel': 0.992272, 'acc_consonant': 0.99093, 'loss_grapheme': 0.227628, 'loss_vowel': 0.170546, 'loss_consonant': 0.114405}
    3 | 0.000037 | 135168/160596 | 1.8368 | 2.2458 |
val: {'recall': 0.98319, 'recall_grapheme': 0.97552, 'recall_vowel': 0.989779, 'recall_consonant': 0.991943, 'acc_grapheme': 0.975947, 'acc_vowel': 0.992073, 'acc_consonant': 0.991055, 'loss_grapheme': 0.178566, 'loss_vowel': 0.130174, 'loss_consonant': 0.096297}
    5 | 0.000034 | 020480/160596 | 1.7801 | 2.2504 |
val: {'recall': 0.982983, 'recall_grapheme': 0.974734, 'recall_vowel': 0.991155, 'recall_consonant': 0.99131, 'acc_grapheme': 0.975574, 'acc_vowel': 0.992471, 'acc_consonant': 0.990955, 'loss_grapheme': 0.207346, 'loss_vowel': 0.135512, 'loss_consonant': 0.102086}
    6 | 0.000030 | 065536/160596 | 1.4792 | 2.1

   35 | 0.000034 | 143360/160596 | 2.4303 | 2.2059 |
val: {'recall': 0.983978, 'recall_grapheme': 0.976505, 'recall_vowel': 0.991757, 'recall_consonant': 0.991143, 'acc_grapheme': 0.976717, 'acc_vowel': 0.993067, 'acc_consonant': 0.991552, 'loss_grapheme': 0.211782, 'loss_vowel': 0.154665, 'loss_consonant': 0.110613}
   37 | 0.000030 | 028672/160596 | 3.3289 | 2.2846 |
val: {'recall': 0.983313, 'recall_grapheme': 0.975249, 'recall_vowel': 0.990985, 'recall_consonant': 0.991769, 'acc_grapheme': 0.975226, 'acc_vowel': 0.992645, 'acc_consonant': 0.991328, 'loss_grapheme': 0.221947, 'loss_vowel': 0.16989, 'loss_consonant': 0.116109}
   38 | 0.000026 | 073728/160596 | 2.3663 | 2.0064 |
val: {'recall': 0.983726, 'recall_grapheme': 0.975903, 'recall_vowel': 0.991326, 'recall_consonant': 0.991774, 'acc_grapheme': 0.97617, 'acc_vowel': 0.993067, 'acc_consonant': 0.991527, 'loss_grapheme': 0.195528, 'loss_vowel': 0.148875, 'loss_consonant': 0.099527}
   39 | 0.000021 | 118784/160596 | 3.7305 | 2

** saved
   70 | 0.000021 | 081920/160596 | 2.3546 | 2.2292 |
val: {'recall': 0.984699, 'recall_grapheme': 0.977967, 'recall_vowel': 0.991848, 'recall_consonant': 0.991015, 'acc_grapheme': 0.976792, 'acc_vowel': 0.992893, 'acc_consonant': 0.99175, 'loss_grapheme': 0.238455, 'loss_vowel': 0.189144, 'loss_consonant': 0.130037}
   71 | 0.000015 | 126976/160596 | 4.1406 | 2.1631 |
val: {'recall': 0.984664, 'recall_grapheme': 0.977666, 'recall_vowel': 0.992009, 'recall_consonant': 0.991316, 'acc_grapheme': 0.976469, 'acc_vowel': 0.99339, 'acc_consonant': 0.991701, 'loss_grapheme': 0.217698, 'loss_vowel': 0.167465, 'loss_consonant': 0.115071}
   73 | 0.000011 | 012288/160596 | 1.4217 | 1.9212 |
val: {'recall': 0.984486, 'recall_grapheme': 0.976901, 'recall_vowel': 0.992169, 'recall_consonant': 0.991972, 'acc_grapheme': 0.977115, 'acc_vowel': 0.993291, 'acc_consonant': 0.991651, 'loss_grapheme': 0.184145, 'loss_vowel': 0.136103, 'loss_consonant': 0.097535}
   74 | 0.000007 | 057344/160596 | 1

...

  639 | 0.000020 | 118784/160596 | 1.6349 | 1.7678 |
val: {'recall': 0.98789, 'recall_grapheme': 0.981731, 'recall_vowel': 0.994271, 'recall_consonant': 0.993827, 'acc_grapheme': 0.980966, 'acc_vowel': 0.995155, 'acc_consonant': 0.993887, 'loss_grapheme': 0.189931, 'loss_vowel': 0.126899, 'loss_consonant': 0.089224}
  641 | 0.000026 | 004096/160596 | 2.0462 | 1.1884 |
val: {'recall': 0.98844, 'recall_grapheme': 0.982713, 'recall_vowel': 0.994836, 'recall_consonant': 0.993498, 'acc_grapheme': 0.982507, 'acc_vowel': 0.995105, 'acc_consonant': 0.994036, 'loss_grapheme': 0.186875, 'loss_vowel': 0.118521, 'loss_consonant': 0.084602}
  642 | 0.000030 | 049152/160596 | 1.4184 | 1.7361 |
val: {'recall': 0.988452, 'recall_grapheme': 0.983364, 'recall_vowel': 0.993732, 'recall_consonant': 0.993347, 'acc_grapheme': 0.982407, 'acc_vowel': 0.994732, 'acc_consonant': 0.994185, 'loss_grapheme': 0.228523, 'loss_vowel': 0.158478, 'loss_consonant': 0.1043}
  643 | 0.000034 | 094208/160596 | 1.3874 | 1.6

  673 | 0.000030 | 012288/160596 | 3.2931 | 1.9145 |
val: {'recall': 0.987336, 'recall_grapheme': 0.981351, 'recall_vowel': 0.993475, 'recall_consonant': 0.993165, 'acc_grapheme': 0.981761, 'acc_vowel': 0.994682, 'acc_consonant': 0.994036, 'loss_grapheme': 0.226451, 'loss_vowel': 0.151893, 'loss_consonant': 0.104403}
  674 | 0.000034 | 057344/160596 | 2.8590 | 1.8665 |
val: {'recall': 0.987459, 'recall_grapheme': 0.981494, 'recall_vowel': 0.993516, 'recall_consonant': 0.993331, 'acc_grapheme': 0.981215, 'acc_vowel': 0.994608, 'acc_consonant': 0.993415, 'loss_grapheme': 0.253049, 'loss_vowel': 0.194104, 'loss_consonant': 0.128449}
  675 | 0.000037 | 102400/160596 | 1.8006 | 1.7673 |
val: {'recall': 0.98811, 'recall_grapheme': 0.98267, 'recall_vowel': 0.99387, 'recall_consonant': 0.993228, 'acc_grapheme': 0.983327, 'acc_vowel': 0.99503, 'acc_consonant': 0.994285, 'loss_grapheme': 0.182617, 'loss_vowel': 0.131624, 'loss_consonant': 0.086898}
  676 | 0.000039 | 147456/160596 | 1.6012 | 1.8

  706 | 0.000037 | 065536/160596 | 1.0108 | 1.7216 |
val: {'recall': 0.988443, 'recall_grapheme': 0.983187, 'recall_vowel': 0.993964, 'recall_consonant': 0.993435, 'acc_grapheme': 0.982929, 'acc_vowel': 0.995204, 'acc_consonant': 0.994061, 'loss_grapheme': 0.200537, 'loss_vowel': 0.124307, 'loss_consonant': 0.083987}
  707 | 0.000039 | 110592/160596 | 1.5843 | 1.7873 |
val: {'recall': 0.987983, 'recall_grapheme': 0.982313, 'recall_vowel': 0.994801, 'recall_consonant': 0.992507, 'acc_grapheme': 0.981861, 'acc_vowel': 0.99513, 'acc_consonant': 0.993763, 'loss_grapheme': 0.25172, 'loss_vowel': 0.164127, 'loss_consonant': 0.109316}
  708 | 0.000040 | 155648/160596 | 0.8717 | 1.7045 |
val: {'recall': 0.98846, 'recall_grapheme': 0.983439, 'recall_vowel': 0.993686, 'recall_consonant': 0.993274, 'acc_grapheme': 0.98278, 'acc_vowel': 0.994856, 'acc_consonant': 0.993962, 'loss_grapheme': 0.200407, 'loss_vowel': 0.126849, 'loss_consonant': 0.089851}
  710 | 0.000039 | 040960/160596 | 0.3669 | 2.0

  739 | 0.000040 | 118784/160596 | 2.6032 | 1.7780 |
val: {'recall': 0.988166, 'recall_grapheme': 0.982743, 'recall_vowel': 0.993662, 'recall_consonant': 0.993515, 'acc_grapheme': 0.981364, 'acc_vowel': 0.994832, 'acc_consonant': 0.993664, 'loss_grapheme': 0.230915, 'loss_vowel': 0.15203, 'loss_consonant': 0.103295}
  741 | 0.000039 | 004096/160596 | 1.1264 | 2.0576 |
val: {'recall': 0.98794, 'recall_grapheme': 0.982222, 'recall_vowel': 0.993804, 'recall_consonant': 0.99351, 'acc_grapheme': 0.981513, 'acc_vowel': 0.994881, 'acc_consonant': 0.993838, 'loss_grapheme': 0.200256, 'loss_vowel': 0.128488, 'loss_consonant': 0.083186}
  741 | 0.000037 | 114688/160596 | 3.3390 | 1.7728 |

KeyboardInterrupt: 

In [ ]:
train(args)

{'recall': 0.978358, 'recall_grapheme': 0.967606, 'recall_vowel': 0.987796, 'recall_consonant': 0.990422, 'acc_grapheme': 0.967374, 'acc_vowel': 0.989241, 'acc_consonant': 0.987998, 'loss_grapheme': 0.33989, 'loss_vowel': 0.214178, 'loss_consonant': 0.145039}
    1 | 0.000040 | 045056/160596 | 4.0782 | 3.0389 |
val: {'recall': 0.97827, 'recall_grapheme': 0.967657, 'recall_vowel': 0.987124, 'recall_consonant': 0.99064, 'acc_grapheme': 0.967225, 'acc_vowel': 0.988918, 'acc_consonant': 0.987576, 'loss_grapheme': 0.370762, 'loss_vowel': 0.25688, 'loss_consonant': 0.164513}
    2 | 0.000039 | 090112/160596 | 5.0198 | 2.5940 |
val: {'recall': 0.976955, 'recall_grapheme': 0.965596, 'recall_vowel': 0.988205, 'recall_consonant': 0.988424, 'acc_grapheme': 0.966927, 'acc_vowel': 0.989241, 'acc_consonant': 0.987973, 'loss_grapheme': 0.266497, 'loss_vowel': 0.173287, 'loss_consonant': 0.117974}
    3 | 0.000037 | 135168/160596 | 0.2808 | 2.7450 |
val: {'recall': 0.978897, 'recall_grapheme': 0.96761

   33 | 0.000039 | 053248/160596 | 1.0196 | 2.7549 |
val: {'recall': 0.979197, 'recall_grapheme': 0.968548, 'recall_vowel': 0.989231, 'recall_consonant': 0.990463, 'acc_grapheme': 0.970232, 'acc_vowel': 0.990558, 'acc_consonant': 0.989216, 'loss_grapheme': 0.269461, 'loss_vowel': 0.194371, 'loss_consonant': 0.132939}
   34 | 0.000037 | 098304/160596 | 1.2563 | 2.5426 |
val: {'recall': 0.979079, 'recall_grapheme': 0.968473, 'recall_vowel': 0.989393, 'recall_consonant': 0.989979, 'acc_grapheme': 0.96961, 'acc_vowel': 0.990185, 'acc_consonant': 0.988868, 'loss_grapheme': 0.277417, 'loss_vowel': 0.199627, 'loss_consonant': 0.146769}
   35 | 0.000034 | 143360/160596 | 1.6984 | 2.5205 |
val: {'recall': 0.978954, 'recall_grapheme': 0.969456, 'recall_vowel': 0.988736, 'recall_consonant': 0.988169, 'acc_grapheme': 0.970082, 'acc_vowel': 0.990061, 'acc_consonant': 0.988619, 'loss_grapheme': 0.258687, 'loss_vowel': 0.163272, 'loss_consonant': 0.114384}
   37 | 0.000030 | 028672/160596 | 3.7338 | 

   66 | 0.000034 | 106496/160596 | 4.7684 | 2.3626 |
val: {'recall': 0.980162, 'recall_grapheme': 0.971361, 'recall_vowel': 0.98908, 'recall_consonant': 0.988848, 'acc_grapheme': 0.97217, 'acc_vowel': 0.991477, 'acc_consonant': 0.989265, 'loss_grapheme': 0.241574, 'loss_vowel': 0.176307, 'loss_consonant': 0.122269}
   67 | 0.000030 | 151552/160596 | 0.2235 | 2.5852 |
val: {'recall': 0.9805, 'recall_grapheme': 0.971908, 'recall_vowel': 0.989568, 'recall_consonant': 0.988616, 'acc_grapheme': 0.972369, 'acc_vowel': 0.99175, 'acc_consonant': 0.989589, 'loss_grapheme': 0.236197, 'loss_vowel': 0.168501, 'loss_consonant': 0.116279}
   69 | 0.000026 | 036864/160596 | 3.6408 | 2.5842 |
val: {'recall': 0.980308, 'recall_grapheme': 0.971174, 'recall_vowel': 0.989555, 'recall_consonant': 0.989331, 'acc_grapheme': 0.971847, 'acc_vowel': 0.99175, 'acc_consonant': 0.989986, 'loss_grapheme': 0.258804, 'loss_vowel': 0.188044, 'loss_consonant': 0.133333}
   70 | 0.000021 | 081920/160596 | 1.7791 | 2.539

   99 | 0.000026 | 159744/160596 | 2.7826 | 2.4088 |
val: {'recall': 0.981572, 'recall_grapheme': 0.972929, 'recall_vowel': 0.990296, 'recall_consonant': 0.990135, 'acc_grapheme': 0.973338, 'acc_vowel': 0.992024, 'acc_consonant': 0.990284, 'loss_grapheme': 0.26153, 'loss_vowel': 0.198119, 'loss_consonant': 0.149081}
  101 | 0.000021 | 045056/160596 | 0.9083 | 2.5024 |
val: {'recall': 0.982115, 'recall_grapheme': 0.973911, 'recall_vowel': 0.990267, 'recall_consonant': 0.990374, 'acc_grapheme': 0.974158, 'acc_vowel': 0.992073, 'acc_consonant': 0.99021, 'loss_grapheme': 0.232243, 'loss_vowel': 0.167849, 'loss_consonant': 0.117313}
  102 | 0.000015 | 090112/160596 | 3.7902 | 2.4828 |
val: {'recall': 0.982729, 'recall_grapheme': 0.975183, 'recall_vowel': 0.989947, 'recall_consonant': 0.990602, 'acc_grapheme': 0.975052, 'acc_vowel': 0.991899, 'acc_consonant': 0.990309, 'loss_grapheme': 0.283059, 'loss_vowel': 0.20083, 'loss_consonant': 0.142793}
** saved
  103 | 0.000011 | 135168/160596 | 2.

  133 | 0.000015 | 053248/160596 | 1.1055 | 2.3244 |
val: {'recall': 0.982106, 'recall_grapheme': 0.973416, 'recall_vowel': 0.990781, 'recall_consonant': 0.990809, 'acc_grapheme': 0.974729, 'acc_vowel': 0.992421, 'acc_consonant': 0.991253, 'loss_grapheme': 0.211503, 'loss_vowel': 0.166163, 'loss_consonant': 0.111303}
  134 | 0.000011 | 098304/160596 | 2.0396 | 2.2117 |
val: {'recall': 0.982572, 'recall_grapheme': 0.974751, 'recall_vowel': 0.990227, 'recall_consonant': 0.990558, 'acc_grapheme': 0.975326, 'acc_vowel': 0.992297, 'acc_consonant': 0.991154, 'loss_grapheme': 0.219274, 'loss_vowel': 0.157781, 'loss_consonant': 0.109228}
  135 | 0.000007 | 143360/160596 | 3.6337 | 2.3152 |
val: {'recall': 0.982873, 'recall_grapheme': 0.974579, 'recall_vowel': 0.990673, 'recall_consonant': 0.991662, 'acc_grapheme': 0.975027, 'acc_vowel': 0.992446, 'acc_consonant': 0.99098, 'loss_grapheme': 0.202821, 'loss_vowel': 0.147704, 'loss_consonant': 0.105511}
  137 | 0.000004 | 028672/160596 | 2.3300 | 

  165 | 0.000007 | 130048/160596 | 0.0454 | 2.0883 |

In [ ]:
#save_model(model, model_file)

In [40]:
del model